In [1]:
import pandas as pd

### 1. Prepare and Load Banner Information

1. load csv file.
2. Fill Null values to Unknown
3. Rename column according to database column names.
4. Save it to banner.csv

In [2]:
imp_columns = ['Academic Year', 'Term', 'Level', 
       'Course', 'Title', 'Primary Instructor Name',
       'Site', 'Banner Schedule Type', 'Meeting Date(s)', 'Meeting Location',
       'Meeting Day(s)', 'Meeting Time', 'Course Section'] #, 'C.R.N.'  

banner = pd.read_csv('historical_scheduleoffering_ET.csv', usecols=imp_columns)
banner['Meeting Location'] = banner['Meeting Location'].fillna(banner['Banner Schedule Type'])
banner['Primary Instructor Name'] = banner['Primary Instructor Name'].fillna('Unknown')
banner['Meeting Date(s)'] = banner['Meeting Date(s)'].fillna('Unknown')
banner['Meeting Day(s)'] = banner['Meeting Day(s)'].fillna('Unknown')
banner['Meeting Time'] = banner['Meeting Time'].fillna('Unknown')

banner.rename(columns = {'Course Section':'course_section','Academic Year':'sch_year', 'Term':'sch_semester','Level':'course_level' , 'Course': 'course_code', 'Title':'course_name',
       'Primary Instructor Name':'primary_instr_name', 'Site':'course_site', 'Banner Schedule Type':'course_type',
       'Meeting Date(s)':'sch_dateslot', 'Meeting Location':'course_meeting_location', 'Meeting Day(s)':'sch_days',
       'Meeting Time':'sch_timeslot' }, inplace = True) 

##Data from historical[  'course_description', 'course_notes', 'course_last_updated','updatedby_name', ]

searchfor = ['CISC', 'STAT']
banner = banner[~banner.course_code.str.contains('|'.join(searchfor))]
searchf = ['Fall 2018', 'Spring 2019', 'J-Term 2019', 'Fall 2019',
       'Summer 2019','J-Term 2020', 'Summer 2020']
banner = banner[~banner.sch_semester.str.contains('|'.join(searchf))]
banner.dropna(inplace=True)
banner.to_csv("banner.csv", index=False)
banner.head(10)

,sch_year,sch_semester,course_level,course_code,course_name,course_section,primary_instr_name,course_site,course_type,sch_dateslot,course_meeting_location,sch_days,sch_timeslot
777,2019-20,Spring 2020,Graduate,EGED530,Fundamentals of ENGR for EDUC,01,"Besser, Deborah M.",St Paul,(B26) Face-to-Face 26-50% of time,4/30-4/30,FDC 317,R,1730-2030
778,2019-20,Spring 2020,Graduate,EGED530,Fundamentals of ENGR for EDUC,01,"Besser, Deborah M.",St Paul,(B26) Face-to-Face 26-50% of time,4/16-4/16,FDC 317,R,1730-2030
779,2019-20,Spring 2020,Graduate,EGED530,Fundamentals of ENGR for EDUC,01,"Besser, Deborah M.",St Paul,(B26) Face-to-Face 26-50% of time,4/2-4/2,FDC 317,R,1730-2030
780,2019-20,Spring 2020,Graduate,EGED530,Fundamentals of ENGR for EDUC,01,"Besser, Deborah M.",St Paul,(B26) Face-to-Face 26-50% of time,3/19-3/19,FDC 317,R,1730-2030
781,2019-20,Spring 2020,Graduate,EGED530,Fundamentals of ENGR for EDUC,01,"Besser, Deborah M.",St Paul,(B26) Face-to-Face 26-50% of time,3/5-3/5,FDC 317,R,1730-2030
782,2019-20,Spring 2020,Graduate,EGED530,Fundamentals of ENGR for EDUC,01,"Besser, Deborah M.",St Paul,(B26) Face-to-Face 26-50% of time,2/20-2/20,FDC 317,R,1730-2030
783,2019-20,Spring 2020,Graduate,EGED530,Fundamentals of ENGR for EDUC,01,"Besser, Deborah M.",St Paul,(B26) Face-to-Face 26-50% of time,2/6-2/6,FDC 317,R,1730-2030
786,2019-20,Spring 2020,Graduate,EGED530,Fundamentals of ENGR for EDUC,01,"Besser, Deborah M.",St Paul,(B26) Face-to-Face 26-50% of time,2/5-5/18,(B26) Face-to-Face 26-50% of time,Unknown,Unknown
788,2019-20,Spring 2020,Graduate,ETLS506,Statistic Methods for Mfg Qlty,01,"Keenan, Thomas M.",St Paul,(LEC) Lecture,2/3-5/18,OWS LL54,R,1745-2045
789,2019-20,Spring 2020,Graduate,ETLS520,Des & Mfg in Med Device Ind,01,"Murray, James C.",St Paul,(LEC) Lecture,2/3-5/18,OSS LL18,W,1745-2045


In [3]:
banner_Nan = banner[banner.isna().any(axis=1)]
banner_Nan.head(10)

,sch_year,sch_semester,course_level,course_code,course_name,course_section,primary_instr_name,course_site,course_type,sch_dateslot,course_meeting_location,sch_days,sch_timeslot


In [4]:
banner.sch_semester.unique()

array(['Spring 2020'], dtype=object)

In [5]:
#nodup = banner[banner.duplicated(['course_code', 'course_section'], keep=False)]

# Select duplicate rows except first occurrence based on all columns
#dup = banner[['course_code', 'course_section', 'course_meeting_location', 'sch_dateslot', 'sch_timeslot']]
duplicateRowsDF = banner[banner.duplicated(['course_code', 'course_section', 'course_meeting_location'])]
aggregated = duplicateRowsDF.groupby(['sch_year', 'sch_semester', 'course_level', 'course_code',
       'course_name', 'course_section', 'primary_instr_name', 'course_site',
       'course_type','course_meeting_location', 'sch_days']).agg({'sch_dateslot':'sum', 'sch_timeslot':'first'}).reset_index()

new_banner = banner.drop_duplicates(['course_code', 'course_section', 'course_meeting_location'])
print("Duplicate Rows except first occurrence based on all columns are :", duplicateRowsDF.shape)
print("Number of courses:", banner.shape[0])
print(new_banner.shape[0])
final_banner = pd.concat([
    new_banner,aggregated
], ignore_index=True)

print(final_banner.shape[0])

Duplicate Rows except first occurrence based on all columns are : (32, 13)
Number of courses: 252
220
227


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [6]:
final_banner.head()

,course_code,course_level,course_meeting_location,course_name,course_section,course_site,course_type,primary_instr_name,sch_dateslot,sch_days,sch_semester,sch_timeslot,sch_year
0,EGED530,Graduate,FDC 317,Fundamentals of ENGR for EDUC,01,St Paul,(B26) Face-to-Face 26-50% of time,"Besser, Deborah M.",4/30-4/30,R,Spring 2020,1730-2030,2019-20
1,EGED530,Graduate,(B26) Face-to-Face 26-50% of time,Fundamentals of ENGR for EDUC,01,St Paul,(B26) Face-to-Face 26-50% of time,"Besser, Deborah M.",2/5-5/18,Unknown,Spring 2020,Unknown,2019-20
2,ETLS506,Graduate,OWS LL54,Statistic Methods for Mfg Qlty,01,St Paul,(LEC) Lecture,"Keenan, Thomas M.",2/3-5/18,R,Spring 2020,1745-2045,2019-20
3,ETLS520,Graduate,OSS LL18,Des & Mfg in Med Device Ind,01,St Paul,(LEC) Lecture,"Murray, James C.",2/3-5/18,W,Spring 2020,1745-2045,2019-20
4,ETLS505,Graduate,OWS 257,Mgr'l Acct & Perform Mgmt,01,St Paul,(LEC) Lecture,"Becker, Thomas W.",2/3-5/18,R,Spring 2020,1745-2045,2019-20


In [7]:
print("Null Values in Banner Data:")
final_banner.isnull().sum()

Null Values in Banner Data:


course_code                0
course_level               0
course_meeting_location    0
course_name                0
course_section             0
course_site                0
course_type                0
primary_instr_name         0
sch_dateslot               0
sch_days                   0
sch_semester               0
sch_timeslot               0
sch_year                   0
dtype: int64

### 3. Load Historical Request data and combine it to banner 

In [8]:
##combine historical requests
##Data from historical[  'course_description', 'course_notes', 'course_last_updated','updatedby_name', ]

hist_columns = ['Course', 'Updatedby', 'Required_4Students', 'Course_Description']
hdf = pd.read_csv('HistoricalRequests.csv',usecols = hist_columns, encoding = "ISO-8859-1")
hdf.head(10)

,Course,Updatedby,Required_4Students,Course_Description
0,ENGR150,"Ling, Tiffany D.",Arduino,This course introduces students to engineering...
1,ENGR160,"Lederle, Rita E.",ARCGIS,Introduction to surveying including the identi...
2,ENGR171,"Willoughby, Patrick J.","SolidWorks, NetSupport School","Through a combination of lectures, hands-on co..."
3,ENGR220,"Baxter, Sarah C.",MASTAN2/Matlab(Req),Principles of statics including such topics as...
4,ENGR221,NaN,MASTAN2/Matlab(Req),Principles of deformable body mechanics includ...
5,ENGR221,"You, Jeong H.","ANSYS - Multiphysics, MASTAN2/Matlab(Req)",Mechanics of Materials Lab
6,ENGR230,"Salamy, Hassan A.","LabView/Multisim (NI), Xilinx - Vivado",Introduction to the design of digital logic. T...
7,ENGR240,"Nepal, Kundan, Min, Cheol-Hong","LabView/Multisim (NI), Matlab/Simulink",Introduction to linear circuit analysis and ba...
8,ENGR240,NaN,"EE Lab, LabView/Multisim (NI), Matlab/Simulink",Circuit Analysis Lab
9,ENGR320,NaN,SolidWorks,"Focus is on advanced mechanics topics, failure..."


In [9]:
import datetime
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M:%S"))

2020-01-30 14:18:37


In [10]:
res = final_banner.merge(hdf, how='left', left_on=['course_code'], right_on=['Course'])

res.rename(columns = {'Course': 'course_code_hist', 'Updatedby':'updatedby_name',
       'Required_4Students':'course_notes', 'Course_Description':'course_description'}, inplace = True)

res['course_code_hist'] = res['course_code_hist'].fillna('Unknown')
res['updatedby_name'] = res['updatedby_name'].fillna('Unknown')
res['course_notes'] = res['course_notes'].fillna('Not Yet Updated.')
res['course_description'] = res['course_description'].fillna('No Description Yet.')
res = res.reset_index()
res['id'] = res.index + 1
res['course_last_updated'] = now.strftime("%Y-%m-%d %H:%M:%S")
res.drop(['course_code_hist', 'index'], axis=1, inplace=True)
##rearrange columns according to database
res = res[['id','course_name', 'course_description', 'course_type', 'course_level', 'course_notes','course_last_updated', 
          'sch_days','sch_timeslot', 'sch_semester','sch_year','course_code', 'course_meeting_location'
          ,'course_site', 'primary_instr_name', 'updatedby_name','course_section','sch_dateslot']]

In [11]:
print("Left Shape:", banner.shape)
print("Right Shape:", hdf.shape)
print("Left Merge Shape:", res.shape)
res.tail()

Left Shape: (252, 13)
Right Shape: (109, 4)
Left Merge Shape: (286, 18)


,id,course_name,course_description,course_type,course_level,course_notes,course_last_updated,sch_days,sch_timeslot,sch_semester,sch_year,course_code,course_meeting_location,course_site,primary_instr_name,updatedby_name,course_section,sch_dateslot
281,282,Requirements Analysis,The objective of this course is to introduce t...,(LEC) Lecture,Graduate,Not Yet Updated.,2020-01-30 14:18:37,S,0900-1600,Spring 2020,2019-20,SEIS636,OSS 329,St Paul,"Gardner, Jan M.",Unknown,01,2/29-2/293/14-3/143/28-3/284/18-4/185/2-5/25/1...
282,283,Machine Learning,Machine Learning builds computational systems ...,(LEC) Lecture,Graduate,Python - Anaconda,2020-01-30 14:18:37,S,0900-1600,Spring 2020,2019-20,SEIS763,OSS 313,St Paul,"Rege, Manjeet","Rege, Manjeet",02,2/29-2/293/14-3/143/28-3/284/18-4/185/2-5/25/1...
283,284,Fabrication Skills Lab,No Description Yet.,(LAB) Lab,Undergraduate,Not Yet Updated.,2020-01-30 14:18:37,R,1630-1930,Spring 2020,2019-20,ENGR255,(LAB) Lab,St Paul,"Wold, Richard C.",Unknown,59,3/30-4/10
284,285,Dynamics Xfer LAB,Principles of dynamics including such topics a...,(LAT) Transfer Lab,Undergraduate,Matlab/Simulink,2020-01-30 14:18:37,T,1330-1510,Spring 2020,2019-20,ENGR322,(LAT) Transfer Lab,St Paul,Unknown,"Holte, Jennifer E.",71,2/3-3/20
285,286,Dynamics Xfer LAB,Dynamics Lab,(LAT) Transfer Lab,Undergraduate,Matlab/Simulink,2020-01-30 14:18:37,T,1330-1510,Spring 2020,2019-20,ENGR322,(LAT) Transfer Lab,St Paul,Unknown,Unknown,71,2/3-3/20


In [12]:
res.primary_instr_name.unique()

array(['Besser, Deborah M.', 'Keenan, Thomas M.', 'Murray, James C.',
       'Becker, Thomas W.', 'Abraham, John P.', 'Beck, Anton F.',
       'Monson, Robert J.', 'Braziller, Sara E.', 'Mahmoodi, Abolghassem',
       'Forliti, David J.', 'Salamy, Hassan A.', 'Unknown',
       'Holmes, Kathleen', 'Min, Cheol-Hong',
       'Nelson-Cheeseman, Brittany B.', 'Lucas, George F.',
       'Albright, F. J.', 'Freedman, Frank B.', 'Kabalan, Mahmoud',
       'Chakravarty, Ranjan K.', 'Mowry, Greg S.',
       'Keyport, Georgiann M.', 'Greene, Christopher S.',
       'Bach, Robert J.', 'Nepal, Kundan', 'Peterson, Kyle T.',
       'Winegar, Michael P.', 'Dorin, Michael A.', 'Rege, Manjeet',
       'Glancy, Aran W.', 'Haug, Frank S.', 'Level, Eric V.',
       'Chiang, Chi-Lung', 'Kazemzadeh, Ebrahim F.', 'Gardner, Jan M.',
       'Berosik, Gary L.', 'Barlow, Jordan B.', 'Naqvi, Syed H.',
       'Gamble, William H.', 'Aune, Aric B.', 'Williams, Timothy K.',
       'Skochil, Jeffrey R.', 'Wallerstedt, 

In [13]:
res.isnull().sum()

id                         0
course_name                0
course_description         0
course_type                0
course_level               0
course_notes               0
course_last_updated        0
sch_days                   0
sch_timeslot               0
sch_semester               0
sch_year                   0
course_code                0
course_meeting_location    0
course_site                0
primary_instr_name         0
updatedby_name             0
course_section             0
sch_dateslot               0
dtype: int64

In [14]:
res.to_csv("banner_historical_merged.csv", index=False)

### 4. Get Instructor Id and Course Id combinations to build database many to many field

In [15]:
## Combine Course with Users
course_columns = ['id', 'course_name', 'course_code', 'primary_instr_name']
users_columns = ['id', 'first_name', 'last_name']

course_new = pd.read_csv('banner_historical_merged.csv', usecols=course_columns, encoding = "ISO-8859-1")
users = pd.read_csv('users.csv', usecols=users_columns , encoding = "ISO-8859-1")

In [16]:
course_new.shape

(286, 4)

In [17]:
users.shape

(29201, 3)

In [18]:
users.isnull().sum()

id             0
first_name    31
last_name     76
dtype: int64

In [19]:
users_Nan = users[users.isna().any(axis=1)]
users_Nan.head(10)

,id,first_name,last_name
500,533,NaN,NaN
718,751,NaN,NaN
1290,1323,craptest2,NaN
1735,1768,NaN,NaN
1852,1885,irasharepoint,NaN
2534,2567,NaN,NaN
3231,3264,NaN,NaN
3477,3510,Institutional Research,NaN
3593,3626,Enrollment Scheduled Reports,NaN
3989,4022,NaN,NaN


In [20]:
course_new.head()

,id,course_name,course_code,primary_instr_name
0,1,Fundamentals of ENGR for EDUC,EGED530,"Besser, Deborah M."
1,2,Fundamentals of ENGR for EDUC,EGED530,"Besser, Deborah M."
2,3,Statistic Methods for Mfg Qlty,ETLS506,"Keenan, Thomas M."
3,4,Des & Mfg in Med Device Ind,ETLS520,"Murray, James C."
4,5,Mgr'l Acct & Perform Mgmt,ETLS505,"Becker, Thomas W."


In [21]:
# new data frame with split value columns 
new = course_new['primary_instr_name'].str.split(" ", n = 2, expand = True)

In [22]:
new.head()

,0,1,2
0,"Besser,",Deborah,M.
1,"Besser,",Deborah,M.
2,"Keenan,",Thomas,M.
3,"Murray,",James,C.
4,"Becker,",Thomas,W.


In [23]:
course_new[['last_name','first_name','middle_name']] = course_new['primary_instr_name'].loc[course_new['primary_instr_name'].str.split(' ').str.len() <= 3].str.split(expand=True)
course_new['last_name'] = course_new['last_name'].str.replace(',','')
course_new.head()

,id,course_name,course_code,primary_instr_name,last_name,first_name,middle_name
0,1,Fundamentals of ENGR for EDUC,EGED530,"Besser, Deborah M.",Besser,Deborah,M.
1,2,Fundamentals of ENGR for EDUC,EGED530,"Besser, Deborah M.",Besser,Deborah,M.
2,3,Statistic Methods for Mfg Qlty,ETLS506,"Keenan, Thomas M.",Keenan,Thomas,M.
3,4,Des & Mfg in Med Device Ind,ETLS520,"Murray, James C.",Murray,James,C.
4,5,Mgr'l Acct & Perform Mgmt,ETLS505,"Becker, Thomas W.",Becker,Thomas,W.


In [24]:
course_new.last_name.unique()

array(['Besser', 'Keenan', 'Murray', 'Becker', 'Abraham', 'Beck',
       'Monson', 'Braziller', 'Mahmoodi', 'Forliti', 'Salamy', 'Unknown',
       'Holmes', 'Min', 'Nelson-Cheeseman', 'Lucas', 'Albright',
       'Freedman', 'Kabalan', 'Chakravarty', 'Mowry', 'Keyport', 'Greene',
       'Bach', 'Nepal', 'Peterson', 'Winegar', 'Dorin', 'Rege', 'Glancy',
       'Haug', 'Level', 'Chiang', 'Kazemzadeh', 'Gardner', 'Berosik',
       'Barlow', 'Naqvi', 'Gamble', 'Aune', 'Williams', 'Skochil',
       'Wallerstedt', 'Filloux', 'Mattox', 'Betz', 'Harvey', 'Kruse',
       'Lai', 'Lunke', 'Roy', 'Misra', 'Yarmoluk', 'Duccini', 'Khou',
       'Chaudhry', 'Willoughby', 'Gagnon', 'Thomas', 'Ling', 'Baxter',
       'Koerner', 'Pfahnl', 'Crepeau', 'Wold', 'Albers', 'Acton', 'Tran',
       'Lederle', 'Megersa', 'Naqwi', 'Berrier', 'Shepard', 'Hofeldt',
       'Hennessey', 'Haas', 'Welt', 'Plourde', 'Wentz', 'You',
       'Japuntich', 'Holte', 'Xu', 'Nathan', 'Ware', 'Toussaint',
       'Secord'], dtype=

In [25]:
course_new.drop(['middle_name','course_name', 'course_code', 'primary_instr_name'], axis=1, inplace=True)
course_new[course_new['last_name'] == 'Rege']

,id,last_name,first_name
38,39,Rege,Manjeet
73,74,Rege,Manjeet
80,81,Rege,Manjeet
81,82,Rege,Manjeet
87,88,Rege,Manjeet
282,283,Rege,Manjeet


In [26]:
users[users['last_name'] == 'Rege']

,id,first_name,last_name
16618,16651,Manjeet,Rege
27147,27180,Ruhee,Rege


In [27]:
course_new.isnull().sum()

id             0
last_name      0
first_name    32
dtype: int64

In [28]:
course_new.dropna(inplace=True)
users.dropna(inplace=True)
course_users = course_new.merge(users, how='inner', left_on=[ 'first_name', 'last_name'], right_on=['first_name', 'last_name'])
print("Found Courses and Instructors:", course_users.shape[0])
course_users = course_users.reset_index()
course_users.head()

Found Courses and Instructors: 195


,index,id_x,last_name,first_name,id_y
0,0,5,Becker,Thomas,8474
1,1,7,Becker,Thomas,8474
2,2,10,Becker,Thomas,8474
3,3,20,Becker,Thomas,8474
4,4,6,Abraham,John,10983


In [29]:
course_users[course_users['first_name'] == 'Chih']

,index,id_x,last_name,first_name,id_y
70,70,72,Lai,Chih,6604
71,71,77,Lai,Chih,6604
72,72,79,Lai,Chih,6604


In [30]:
course_users['id'] = course_users.index + 1
course_users.drop(['index','last_name', 'first_name'], axis=1, inplace=True)
course_users.rename(columns = {'id_x': 'course_id', 'id_y':'user_id'}, inplace = True)
course_users = course_users[['id', 'course_id', 'user_id']]
course_users.to_csv("course_users.csv", index=False)
course_users.tail()

,id,course_id,user_id
190,191,277,10811
191,192,278,10811
192,193,267,8229
193,194,274,13538
194,195,275,13538


### 5. Get Classroom Id and Course Id combinations to build database many to many field

-- Check for any missing classroom from the existing list.

In [31]:
##Check Class Differences from Existing list

course_columns = ['id', 'course_meeting_location']
classroom_columns = ['id', 'class_name']

course_class = pd.read_csv('banner_historical_merged.csv', usecols=course_columns, encoding = "ISO-8859-1")
classroom = pd.read_csv('classroom.csv', usecols=classroom_columns , encoding = "ISO-8859-1")
print(list(set(banner.course_meeting_location.unique()) - set(classroom.class_name.unique())))

[]


In [32]:
classroom.head()

,id,class_name
0,1,OSS 318
1,2,OSS 333
2,3,OSS 313
3,4,OSS LL18
4,5,OSS LL13


In [33]:
course_classroom = course_class.merge(classroom, how='inner', left_on=['course_meeting_location'], right_on=['class_name'])
print("Found Courses and Classrooms:", course_classroom.shape[0])

course_classroom = course_classroom.reset_index()
course_classroom['id'] = course_classroom.index + 1
course_classroom.drop(['index','course_meeting_location', 'class_name'], axis=1, inplace=True)
course_classroom.rename(columns = {'id_x': 'course_id', 'id_y':'classroom_id'}, inplace = True)
course_classroom = course_classroom[['id', 'course_id', 'classroom_id']]
course_classroom.to_csv("course_classroom.csv", index=False)
course_classroom.tail()

Found Courses and Classrooms: 286


,id,course_id,classroom_id
281,282,272,39
282,283,277,39
283,284,278,39
284,285,256,61
285,286,273,57


### 6. Get Software Id and Course Id combinations to build database many to many field

In [34]:
##Prepare Software Data for inserting
softwares = pd.read_csv('softwares.csv', encoding = "ISO-8859-1")
softwares['sware_description'] = softwares['sware_description'].fillna('No Description yet.')
softwares['sware_notes'] = softwares['sware_notes'].fillna('No Notes yet.')
softwares.to_csv("softwares.csv", index=False)